## Import libraries

In [94]:
import pandas as pd 
import numpy as np 
import matplotlib as plt
import math
import pprint
from numpy import log2 as log 
from pprint import pprint

import warnings
warnings.filterwarnings('ignore')

## Import Datasets (write the path to the .csv file below)

In [95]:
data=pd.read_csv('C:/Users/Shikhar/Desktop/ML/Asgn-2/data/winequality-red.csv', sep=';')

## Convert all the values in quality attribute to 0 (bad) if the value is less than or equal to ‘6’ and others to 1(good)

In [96]:
x=data['quality']
for i in range(len(x)):
    if(x[i] <5):
        x[i]=0
    elif(x[i]==5 or x[i]==6):
        x[i]=1
    else:
        x[i]=2

## Z-Score Normalization

In [97]:
s='fixed acidity'
data[s]=(data[s]-data[s].mean())/(data[s].std())


s='volatile acidity'
data[s]=(data[s]-data[s].mean())/(data[s].std())

s='citric acid'
data[s]=(data[s]-data[s].mean())/(data[s].std())

s='residual sugar'
data[s]=(data[s]-data[s].mean())/(data[s].std())

s='chlorides'
data[s]=(data[s]-data[s].mean())/(data[s].std())

s='free sulfur dioxide'
data[s]=(data[s]-data[s].mean())/(data[s].std())

s='total sulfur dioxide'
data[s]=(data[s]-data[s].mean())/(data[s].std())

s='density'
data[s]=(data[s]-data[s].mean())/(data[s].std())

s='pH'
data[s]=(data[s]-data[s].mean())/(data[s].std())

s='sulphates'
data[s]=(data[s]-data[s].mean())/(data[s].std())

s='alcohol'
data[s]=(data[s]-data[s].mean())/(data[s].std())

## Segregation of data into 4 equally spaced bins represented as 0,1,2,3

In [98]:
category = pd.cut(data['fixed acidity'], 4, labels=['0','1','2','3'], precision=100)
data.drop(['fixed acidity'], inplace=True, axis=1)
data.insert(0, 'fixed acidity', category)

category = pd.cut(data['volatile acidity'], 4, labels=['0','1','2','3'], precision=100)
data.drop(['volatile acidity'], inplace=True, axis=1)
data.insert(1, 'volatile acidity', category)

category = pd.cut(data['citric acid'], 4, labels=['0','1','2','3'], precision=100)
data.drop(['citric acid'], inplace=True, axis=1)
data.insert(2, 'citric acid', category)

category = pd.cut(data['residual sugar'], 4, labels=['0','1','2','3'], precision=100)
data.drop(['residual sugar'], inplace=True, axis=1)
data.insert(3, 'residual sugar', category)

category = pd.cut(data['chlorides'], 4, labels=['0','1','2','3'], precision=100)
data.drop(['chlorides'], inplace=True, axis=1)
data.insert(4, 'chlorides', category)

category = pd.cut(data['free sulfur dioxide'], 4, labels=['0','1','2','3'], precision=100)
data.drop(['free sulfur dioxide'], inplace=True, axis=1)
data.insert(5, 'free sulfur dioxide', category)

category = pd.cut(data['total sulfur dioxide'], 4, labels=['0','1','2','3'], precision=100)
data.drop(['total sulfur dioxide'], inplace=True, axis=1)
data.insert(6, 'total sulfur dioxide', category)

category = pd.cut(data['density'], 4, labels=['0','1','2','3'], precision=100)
data.drop(['density'], inplace=True, axis=1)
data.insert(7, 'density', category)

category = pd.cut(data['pH'], 4, labels=['0','1','2','3'], precision=100)
data.drop(['pH'], inplace=True, axis=1)
data.insert(8, 'pH', category)

category = pd.cut(data['sulphates'], 4, labels=['0','1','2','3'], precision=100)
data.drop(['sulphates'], inplace=True, axis=1)
data.insert(9, 'sulphates', category)

category = pd.cut(data['alcohol'], 4, labels=['0','1','2','3'], precision=100)
data.drop(['alcohol'], inplace=True, axis=1)
data.insert(10, 'alcohol', category)

In [99]:
y=data['quality']
X=data.drop(['quality'], axis=1)

In [100]:
X=np.array(X)
y=np.array(y)

# Dataset is ready for Decision Tree 

## ID3 Decision tree using Information Gain in Python

In [101]:
#Breaking the dataset into train and test dataset

m=len(y)
train=[]
test=[]
Y=[]
for j in range(m//3,m):
    l=[]
    Y.append(y[j])
    for i in range(0,11):
        l.append(int(X[j][i]))
    l.append(int(y[j]))
    train.append(l)
for j in range(0,m//3):
    l=[]
    for i in range(0,11):
        l.append(int(X[j][i]))
    l.append(int(y[j]))
    test.append(l)
    
dataset=pd.DataFrame(train,columns=['fixed acidity','volatile acidity','citric acid','residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol','quality'])
testdataset=pd.DataFrame(test,columns=['fixed acidity','volatile acidity','citric acid','residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol','quality'])

## Entropy function

In [102]:
def entropy(Column):
    elements,counts = np.unique(Column,return_counts = True)
    m=len(elements)
    for i in range(0,m):
        entropy = np.sum([(-counts[i]/np.sum(counts))*np.log2(counts[i]/np.sum(counts)) for i in range(len(elements))])   #Entropy of each node is found out using the formula taught
    return entropy

## Information Gain function

In [103]:
def InformationGain(data,split_attribute_name,target_name="quality"):
    total_entropy = entropy(data[target_name]) 
    vals,counts= np.unique(data[split_attribute_name],return_counts=True)
    Weighted_Entropy = np.sum([(counts[i]/np.sum(counts))*entropy(data.where(data[split_attribute_name]==vals[i]).dropna()[target_name]) for i in range(len(vals))])
    Information_Gain = total_entropy - Weighted_Entropy      #Info-gain is found put using the entropy function above
    return Information_Gain

## Build Decision Tree function

In [104]:
def Build_Decision_Tree(data,originaldata,features,target_attribute_name="quality",parent_node_class = None):
    if len(np.unique(data[target_attribute_name])) <= 1:
        return np.unique(data[target_attribute_name])[0]
    
    elif len(data)<10:
        return np.unique(originaldata[target_attribute_name])[np.argmax(np.unique(originaldata[target_attribute_name],return_counts=True)[1])]
    
    elif len(features) ==0:
        return parent_node_class    
    else:
        parent_node_class = np.unique(data[target_attribute_name])[np.argmax(np.unique(data[target_attribute_name],return_counts=True)[1])]
        item_values = [InformationGain(data,feature,target_attribute_name) for feature in features] #Return the information gain values for the features in the dataset
        best_feature_index = np.argmax(item_values)
        best_feature = features[best_feature_index]
        tree = {best_feature:{}}
        features = [i for i in features if i != best_feature]        
        for value in np.unique(data[best_feature]):
            value = value
            sub_data = data.where(data[best_feature] == value).dropna()
            subtree = Build_Decision_Tree(sub_data,dataset,features,target_attribute_name,parent_node_class)
            tree[best_feature][value] = subtree
        return(tree)   
    
Tree=Build_Decision_Tree(dataset,dataset,dataset.columns[:-1])

## k-fold cross validation on Decision Tree

In [105]:
# Prediction function for the decision tree

def predict(query,tree):
    for key in list(query.keys()):
        if key in list(tree.keys()):
            try:
                result = tree[key][query[key]] 
            except:
                return 1
            result = tree[key][query[key]]
            if isinstance(result,dict):
                return predict(query,result)
            else:
                return result
queries=testdataset.iloc[:,:-1].to_dict(orient = "records")
count=0

#1st fold cross validation

True_Positive=0
True_Negative=0
False_Positive=0
False_Negative=0
Y=[]
for i in range(0,m//3):
    y_pred=predict(queries[i],Tree)
    Y.append(y_pred)
    if(y_pred==y[i]):
        if(y[i]==1):
            True_Positive+=1
        else:
            True_Negative+=1
    else:
        if(y_pred==1):
            False_Positive+=1
        else:
            False_Negative+=1
            
Accuracy_1=(True_Positive+True_Negative)/(True_Positive+True_Negative+False_Negative+False_Positive)
Precision_1=True_Positive/(True_Positive+False_Positive)
Recall_1=True_Positive/(True_Positive+False_Negative)


#2nd fold cross validation

True_Positive=0
True_Negative=0
False_Positive=0
False_Negative=0
for i in range(0,m//3):
    y_pred=Y[i]
    if(y_pred==y[i]):
        if(y[i]==0):
            True_Positive+=1
        else:
            True_Negative+=1
    else:
        if(y_pred==0):
            False_Positive+=1
        else:
            False_Negative+=1
            
Accuracy_2=(True_Positive+True_Negative)/(True_Positive+True_Negative+False_Negative+False_Positive)
count=0
if(True_Positive+False_Positive==0):
    count+=1
    Precision_2=0
else:
    Precision_2=True_Positive/(True_Positive+False_Positive)
Recall_2=True_Positive/(True_Positive+False_Negative)

#3rd fold cross validation

True_Positive=0
True_Negative=0
False_Positive=0
False_Negative=0
for i in range(0,m//3):
    y_pred=Y[i]
    if(y_pred==y[i]):
        if(y[i]==2):
            True_Positive+=1
        else:
            True_Negative+=1
    else:
        if(y_pred==2):
            False_Positive+=1
        else:
            False_Negative+=1
            
Accuracy_3=(True_Positive+True_Negative)/(True_Positive+True_Negative+False_Negative+False_Positive)
if(True_Positive+False_Positive==0):
    Precision_3=0
    count+=1
else:
    Precision_3=True_Positive/(True_Positive+False_Positive)
Recall_3=True_Positive/(True_Positive+False_Negative)

#Printing results 3-fold cross validation

print("Macro Accuracy from basic decision tree:",(Accuracy_1+Accuracy_2+Accuracy_3)/3)
print("Macro Precision from basic decision tree:",(Precision_1+Precision_2+Precision_3)/(3))
print("Macro Recall from basic decision tree:",(Recall_1+Recall_2+Recall_3)/(3))

Macro Accuracy from basic decision tree: 0.8311444652908068
Macro Precision from basic decision tree: 0.43720930232558136
Macro Recall from basic decision tree: 0.3520767202431938


## Decision Tree building using scikitlearn library

In [106]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

In [107]:
model=DecisionTreeClassifier(criterion='entropy', max_depth=4,  min_samples_split=10)
model.fit(X,y)
y_predict = model.predict(X)
print("Accuracy Score from scikit learn Decision Tree is :", accuracy_score(y,y_predict))
print("Precision Score from scikit learn Decision Tree is :", precision_score(y,y_predict, average='macro'))
print("Recall Score from scikit learn Decision Tree is :", recall_score(y,y_predict, average='macro'))

Accuracy Score from scikit learn Decision Tree is : 0.8480300187617261
Precision Score from scikit learn Decision Tree is : 0.544021444082519
Recall Score from scikit learn Decision Tree is : 0.406852931688462


### k-fold cross validation using scikit learn (k=3)

In [108]:
model=DecisionTreeClassifier(criterion='entropy', max_depth=4,  min_samples_split=10)

print("Accuracy score from scikit learn Decision Tree is :", cross_val_score(model,X,y,cv=3).mean())    #Mean accuracy using k-fold cross validation

Accuracy score from scikit learn Decision Tree is : 0.8311358710314444
